In [3]:
import os
import yaml
with open('chatgpt-api-credentials.yml') as file:
    openai_key = yaml.safe_load(file)
with open('tavily-api-credentials.yml') as file:
    tavilyai_key = yaml.safe_load(file)
with open('weather-api-credentials.yml') as file:
    weather_key = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = openai_key['OPENAI-API-KEY']
os.environ['TAVILY_API_KEY'] = tavilyai_key['TAVILY-API-KEY']
weather_api = weather_key['WEATHER-API-KEY']

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import requests
import json

tv_search = TavilySearchResults(max_results=3,search_depth="advanced",max_tokens=10000)

@tool
def search_web(query: str) -> list:
    """ Search the web for a query """
    tavily_tool = TavilySearchResults(max_results=2)
    results = tavily_tool.invoke(query)
    return results

@tool
def get_weather(query: str) -> list:
    """ Search weatherapi to get weather results """
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={weather_api}&q={query}"
    

In [10]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chatgpt = ChatOpenAI(model="gpt-4o",temperature=0.3)
tools =[search_web,get_weather]

sys_prompt = """
        Act as helpful AI assitant and use the tools that are disposed at you to perform the tasks as needed.
        - get_weather : whenever user asks a query about weather.
        - search_web : whenever user asks about current events or if you dont know the answer.
        """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", sys_prompt),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human","{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chatgpt_with_tools = chatgpt.bind_tools(tools)
agent = create_tool_calling_agent(chatgpt,tools,prompt_template)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [12]:
!rm memory.db

rm: memory.db: No such file or directory


In [14]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id,"sqlite:///memory.db")

agentic_chatbot = RunnableWithMessageHistory(
                    agent_executor,
                    get_session_history_db,
                    input_messages_key="query",
                    history_messages_key="history",
                    )

from IPython.display import Markdown, display
def chat_with_agent(prompt:str,session_id:str):
    response = agentic_chatbot.invoke(
        {"query":prompt},{"configurable":{"session_id":session_id}}
    )
    display(Markdown(response['output']))

In [15]:
user="praveen"
query="Who is naga chaitanya married to?"
chat_with_agent(query,user)

Naga Chaitanya is currently engaged to Sobhita Dhulipala.

In [16]:
user2="vk"
query2="Who is pawan kalyan married to?"
chat_with_agent(query2,user2)

Pawan Kalyan is currently married to Anna Lezhneva.

In [18]:
query3="who did he divorce?"
chat_with_agent(query3,user)
chat_with_agent(query3,user2)

Naga Chaitanya divorced Samantha Ruth Prabhu. They announced their separation in October 2021 after being married for nearly four years.

Pawan Kalyan has been divorced twice. His first marriage was to Nandini, which ended in divorce in 2007. His second marriage was to Renu Desai, which ended in divorce in 2012.